## Lab Six: CNNs
GROUP MEMBERS:
- **Alex Chen** 
- **Paige Maple** 
- **Sam Valentine**

### Sources
1. ChatGPT (For formatting text and plots)

### Preparation (3 pts total)

#### Part One (1.5 pts)

#### Part Two (1.5 pts)

### Modeling (6 pts total)

#### Part One (1.5 pts)

#### Part Two (2 pts)

#### Part Three (1.5 pts)

#### Part Four (1 pts)

### Exceptional Work (1 pts total)
One idea (required for 7000 level students): Use transfer learning with pre-trained weights for your initial layers of your CNN. Compare the performance when using transfer learning to your best model from above in terms of classification performance. 